In [2]:
from glob import glob
import pandas as pd
import rasterio


In [7]:
chips_df = pd.read_csv('/workspace/gfm-gap-filling/pretraining/chip_tracker.csv')
#tifs = glob('/workspace/data/chips_binary_filtered/*_merged.tif')
cdls = glob('/workspace/gfm-gap-filling/pretraining/train_single_band/chips_single_band/*.CDL.tif')

output_folder = ('/workspace/data/chips_single_band/')

In [10]:
cdls[0]

'/workspace/gfm-gap-filling/pretraining/train_single_band/chips_single_band/HLS.S30.chip_049_069.2021101.v1.4.CDL.tif'

In [ ]:
tifs

In [ ]:
## chatgpt function
def export_bands(input_path, output_dir, band_names):
    with rasterio.open(input_path) as src:
        #print(src.count)
        for band_idx in range(src.count):
            # Read the band data
            band = src.read(band_idx + 1)
            
            # Define the output file path
            output_path = f"{output_dir}/{band_names[band_idx]}.tif"
            out_meta = src.meta.copy()
            out_meta.update({"count": 1})
            # Create the output file and write the band data
            with rasterio.open(output_path, 'w', **out_meta) as dst:
                dst.write(band, 1)

# Example usage
# input_path = 'path/to/input.tif'  # Path to the input multiband GeoTIFF
# output_dir = 'path/to/output'     # Directory to save the output TIFF files
# band_names = ['red', 'green', 'blue']  # User-defined names for each band

# export_bands(input_path, output_dir, band_names)


In [13]:
def row_to_dict(data_frame, chip_id):
    # Extract the specified row as a pandas Series
    row = data_frame.loc[data_frame.chip_id == chip_id]
    
    # Convert the row to a dictionary
    row_dict = row.to_dict(orient = 'records')[0]
    
    return row_dict



In [43]:
chips_df.shape

(6400, 16)

In [ ]:
## export cdl binary




for tif_path in cdls:
  #  print(tif_path)
    chip_id = tif_path.split('/')[-1].split('.')[2][5:]
   # print(chip_id)
    chip_index = chips_df.loc[chips_df.chip_id == chip_id].index.values.astype(int)
  #  print(chip_index)

    with rasterio.open(tif_path) as src:
        values = src.read()
     #   print(values.sum())
        cdl_sum = values.sum()
        
        
    
    chips_df.loc[chips_df.chip_id == chip_id, 'CDL_count'] = cdl_sum
   # chip_meta = row_to_dict(chips_df, chip_id[5:])
  #  print(chip_meta)


    
    # date_1 = str(int(chip_meta['first_image_date']))
    # date_2 = str(int(chip_meta['second_image_date']))
    # date_3 = str(int(chip_meta['third_image_date']))
    # band_names = [f'HLS.S30.{chip_id}.{date_1}.v1.4.CDL']
    # export_bands(tif_path, output_folder, band_names)

    # band_names = [f'HLS.S30.{chip_id}.{date_2}.v1.4.CDL']
    # export_bands(tif_path, output_folder, band_names)

    # band_names = [f'HLS.S30.{chip_id}.{date_3}.v1.4.CDL']
    # export_bands(tif_path, output_folder, band_names)
    
    



In [46]:
cdl_0_crop = chips_df[chips_df.CDL_count == 0]
cdl_0_crop.shape

(14, 16)

In [44]:
chips_df.to_csv('/workspace/gfm-gap-filling/pretraining/chip_tracker_cdl_count.csv')